In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings("ignore")

In [3]:
import glob
import librosa
import librosa.display
from IPython.display import Audio, display
import matplotlib.pyplot as plt

In [7]:
def read_path_audio_and_target(is_train=True):
    paths, targets = [], []
    
    if is_train:
        name = 'train'
    else:
        name = 'valid'
        
    for cls in ['student', 'professor']:
        path = glob.glob(f'/media/auishik/New Volume/Personal/kaggle/diarization/archive/dataset/{name}/{cls}/*')
    
        if cls == 'student':
            target = [0 for _ in range(len(path))]
        else:
            target = [1 for _ in range(len(path))]

        paths.extend(path)
        targets.extend(target)
    
    return paths, targets

In [8]:
paths_train, targets_train = read_path_audio_and_target(is_train=True)
paths_test, targets_test = read_path_audio_and_target(is_train=False)

In [9]:
print(paths_train[0])
sample_student, sr = librosa.load(paths_train[0])
display(Audio(sample_student, rate=sr))

/media/auishik/New Volume/Personal/kaggle/diarization/archive/dataset/train/student/audio_1.wav


In [10]:
print(paths_train[-1])
sample_professor, sr = librosa.load(paths_train[-1])
display(Audio(sample_professor, rate=sr))

/media/auishik/New Volume/Personal/kaggle/diarization/archive/dataset/train/professor/audio_7.wav


In [11]:
def reshape_sound_and_target(path_sound, targets, segment_size_t=1):
    """
    
    Parameters:
    path_sound, 
    targets, 
    segment_size_t: segment size in seconds
    """
    segments_sound, segments_target = [], []
    
    for i in range(len(path_sound)):
        # load
        y, sr = librosa.load(path_sound[i])

        signal_len = len(y) 
        # segment size in samples
        segment_size = int(segment_size_t * sr)  
        # Break signal into list of segments in a single-line Python code
        segments = [y[x:x + segment_size] for x in np.arange(0, signal_len, segment_size)]

        target = [targets[i] for _ in range(len(segments))]

        segments_sound.extend(segments)
        segments_target.extend(target)

    segments_sound = np.array(segments_sound)
    segments_target = np.array(segments_target)
    
    return segments_sound, segments_target, sr

In [12]:
segment_size_t = 0.5

segments_sound_train, segments_target_train, sr = reshape_sound_and_target(paths_train, targets_train, segment_size_t)
segments_sound_test, segments_target_test, _ = reshape_sound_and_target(paths_test, targets_test, segment_size_t)